# Basic Examples: communicating through the API


In [ ]:
import raymon
from raymon import RaymonAPI

api = RaymonAPI(url="https://api.raymon.ai/v0")

## Creating or searching a project
Using `api.project_search`, you can look up the project_id given a project name. `api.project_create` allows you to create a new project.

In [ ]:
project_id = None
resp = api.project_search(project_name="raymon-tutorials")
if resp.ok:
    print(f"Project found")
    project_id = resp.json()["project_id"]
else:
    print(f"Creating new project")
    resp = api.project_create(project_name="raymon-tutorials")
    project = resp.json()
    project_id = project["project_id"]

print(f"Project ID: {project_id}")

## Logging to the project using the API logger
In the code snippet below, we create an `RaymonAPILogger` logger first, and then pass it as a parameter when constructing a `Trace` object. We do not pass a `trace_id`, which means a `uuid` will be auto generated. If you alreay have a `uuid`, you can pass it as `trace_id`.

In [ ]:
from raymon import Trace, RaymonAPILogger

logger = RaymonAPILogger(project_id=project_id, batch_size=4)

trace = Trace(logger=logger, trace_id=None) 

## Logging text
You can use the trace like any other logger to log info text messages, as shown below.

In [ ]:
trace.info("You can log whatever you want here")

## Logging tags
It is also possible to attach tags to a given data trace. Tags have a `name`, a `value`, a `type` and an optional tag `group`. Tags can be passed as a list of dicts with the aforementioned keys, or you can use the `raymon.Tag` object.

Tags can be any metric, label or metadata you want to attach to the trace. The Raymon backend allows you to filter data based on these tags and aggregates tags into metrics.

In [ ]:
from raymon import Tag

tags = [
    # Using a dict
    {
        "name": "client",
        "value": "bigshot_client",
        "type": "label"
    },
    # Using the Tag ogbject
    Tag(name="sdk_version", value="1.4.2", type="label"),
    Tag(name="prediction_time_ms", value="120", type="metric")
]
trace.tag(tags)

## Logging data

Raymon allows you to log data artefacts to the backend too. The artefacts have a reference that allows you to fetch them from the backend whenever you want. By default, these artefacts are simply stored, although you can do extra processing on them with some configuration in the project manifest. (Explained in other tutorials.)

Raymon offers data wrappers for popular data types that will take care of serializing your data in the `raymon.types` module. Of course, you can also define your own wrappers if you need them!

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

import raymon.types as rt


img = Image.open("../raymon/tests/sample_data/castinginspection/def_front/cast_def_0_0.jpeg")
arr = np.array([[1, 2], [3, 4]])
df = pd.DataFrame(arr, columns=['a', 'b'])

trace.log(ref="native-reff", data=rt.Native(
    {"foo": "bar", 
     "whatever": ["you", "want"], 
     "all_native_types": 1}))
trace.log(ref="numpy-ref", data=rt.Numpy(arr))
trace.log(ref="pandas-ref", data=rt.DataFrame(df))
trace.log(ref="image-ref", data=rt.Image(img))


In [ ]:
!pwd

## Accessing the current Trace
You can either pass along the Trace object as a parameter in function calls, or create a new Trace object with the relevant `trace_id` anywhere you need it (meaning, you would have to pass along this `id`). Alternatively you can create a Trace object once and use `raymon.current_trace` to get it whenever you need.

In [ ]:
trace2 = raymon.current_trace()
trace2.trace_id

## Fetching data
Using the `trace_id` and artefact `ref`, you can easlity fetch artefacts from the backend and load the data. 

In [ ]:
resp = api.object_search(project_id=project_id, trace_id=trace.trace_id, ref="numpy-ref")

if not resp.ok:
    raise Exception("Something wrong.")

data = resp.json()
obj_id = data["obj_id"]
obj_data = data["obj_data"]

raymon_wrapped = rt.load_jcr(obj_data)
orig = raymon_wrapped.data
orig